In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import scanpy as sc,anndata as ad
import squidpy as sq
import os
import sys
from scipy.sparse import vstack
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,scale
from scipy.spatial import distance_matrix, distance
from sklearn.neighbors import KernelDensity
from scipy.spatial import KDTree
from scipy.spatial.distance import cdist
import networkx as nx
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
from sklearn.neighbors import BallTree
import time
import rdata
from scipy.sparse import csr_matrix,bsr_matrix,coo_matrix,issparse,lil_matrix,diags
from scipy.sparse.linalg import inv
import scipy as sp
from  scipy.ndimage import gaussian_filter
import igraph as ig
import glasbey
import warnings
import cairocffi as cairo
from sklearn.metrics import adjusted_rand_score,make_scorer
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn import preprocessing
import libpysal
from esda.losh import LOSH
from multiprocessing import Pool
from mclustpy import mclustpy
from sklearn.cluster import KMeans
import torch
#from GraphST import GraphST
import SEDR
#from GraphST.utils import clustering
from matplotlib.colors import ListedColormap
from esda import Moran
from libpysal.weights import KNN
from scsampler import scsampler 
from geosketch import gs
from scvalue import SCValue
from tqdm import tqdm
from fbpca import pca

In [8]:
#lets try the subsampling on a 
# need implimentation of Robust hausdorff distance computation 
from scipy.spatial.distance import directed_hausdorff
def partial_hausdorff_distance(array1, array2, q=1e-4):
    """
    Compute the partial Hausdorff distance from array1 to array2.
    
    Parameters:
    - array1: numpy.ndarray, shape (n, d)
        The first array of points (e.g., representing the full dataset).
    - array2: numpy.ndarray, shape (m, d)
        The second array of points (e.g., representing the subset).
    - q: float, optional
        The parameter for the partial Hausdorff distance, should be between 0 and 1.
        q=0 corresponds to the classical Hausdorff distance, q=1 would correspond to
        considering up to the smallest distance, practically ineffective in most uses.
    
    Returns:
    - float
        The partial Hausdorff distance based on q between array1 and array2.
    """
    
    # Calculate distances from each point in array1 to the nearest point in array2
    # distances = [directed_hausdorff([point], array2)[0] for point in tqdm(array1)]
    
    # # Sort the distances
    # distances.sort()
    
    # # Calculate the index for the Kth largest value
    # K = int(np.floor((1 - q) * len(distances)))
    
    # # Return the Kth largest value from the sorted distances
    # return distances[K-1] if K > 0 else distances[0]

    # Build KD-Tree for array2
    print("building KDTree")
    start = time.time()
    tree = KDTree(array2)
    end = time.time()
    print(f"KDTree took {end-start:.4f} seconds to build")
    # Query the tree with all points in array1 to find nearest neighbors efficiently
    print("query tree")
    start = time.time()
    distances, _ = tree.query(array1, k=1)
    end = time.time()
    print(f"distance calculation took: {end-start:.4f} seconds")

    # Sort all computed distances
    distances.sort()

    # Determine the index of the Kth largest value for partial Hausdorff
    K = int(np.floor((1 - q) * len(distances)))

    # Handle edge cases and return the Kth largest value
    return distances[K-1] if K > 0 else distances[0]

def uniform_sample_adata(adata, fraction=0.1, random_state=None):
    """
    Uniformly samples a fraction of cells from an AnnData object.
    
    Parameters:
    - adata: AnnData object
    - fraction: float, fraction of cells to sample (default is 10%)
    - random_state: int, random seed for reproducibility (default is None)
    
    Returns:
    - AnnData object with sampled cells
    """
    np.random.seed(random_state)
    num_cells = adata.n_obs  # Total number of cells
    sample_size = int(num_cells * fraction)  # Compute number of cells to sample

    sampled_indices = np.random.choice(adata.obs.index, size=sample_size, replace=False)
    
    return adata[sampled_indices].copy()  # Return a new AnnData object

def probability_sample_adata(adata, fraction=0.1, score_column="100_highly_variable_gene_score", random_state=None):
    """
    Samples a fraction of cells from an AnnData object using weighted probabilities.

    Parameters:
    - adata: AnnData object
    - fraction: float, fraction of cells to sample (default is 10%)
    - score_column: str, column in adata.obs containing sampling probabilities
    - random_state: int, random seed for reproducibility (default is None)

    Returns:
    - AnnData object with sampled cells
    """
    np.random.seed(random_state)

    if score_column not in adata.obs:
        raise ValueError(f"Column '{score_column}' not found in adata.obs")

    scores = adata.obs[score_column].values
    scores = np.clip(scores, a_min=0, a_max=None)  # Ensure no negative values
    probabilities = scores / scores.sum()  # Normalize to get probabilities

    num_cells = adata.n_obs
    sample_size = int(num_cells * fraction)

    sampled_indices = np.random.choice(adata.obs.index, size=sample_size, replace=False, p=probabilities)

    return adata[sampled_indices].copy()

def evaluate_sampling_methods(adata, fraction=0.1, seed=0, k=20):
    """
    Evaluates different sampling methods for spatial transcriptomics data and computes partial Hausdorff distances.

    Parameters:
    - adata: AnnData object
    - fraction: float, fraction of cells to sample
    - seed: int, random seed for reproducibility
    - k: int, number of principal components for PCA-based geosketching

    Returns:
    - A dictionary containing sampled datasets
    - A pandas DataFrame with computed distances
    """
    start = time.time()

    # Define sampling methods
    sampling_methods = {
        "uniform": lambda adata: uniform_sample_adata(adata, fraction=fraction, random_state=seed),
        "leverage": lambda adata: probability_sample_adata(adata, fraction=fraction, random_state=seed),
        "scsampler_transcriptomic": lambda adata: scsampler(adata, fraction=fraction, random_state=seed, copy=True),
    }

    # Run sampling methods
    sampled_data = {name: method(adata) for name, method in sampling_methods.items()}

    # PCA-based geosketching
    U, s, _ = pca(adata.X, k=k)
    X_dimred = U[:, :k] * s[:k]
    N = int(fraction * adata.X.shape[0])
    sampled_data["geo_transcriptomic"] = adata[gs(X_dimred, N, seed=seed, replace=False)].copy()

    # Coordinate-based sampling
    sampled_data["scsampler_coords"] = adata[scsampler(adata.obsm['spatial'], fraction=fraction, random_state=seed, copy=True)[1]].copy()
    sampled_data["geo_coords"] = adata[gs(adata.obsm['spatial'], N, seed=seed, replace=False)].copy()

    # Compute partial Hausdorff distances
    distance_data = []
    for name, subset in sampled_data.items():
        distance_data.append({
            "method": name,
            "transcriptomic_distance": partial_hausdorff_distance(adata.X.toarray(), subset.X.toarray(), q=1e-4),
            "spatial_distance": partial_hausdorff_distance(adata.obsm['spatial'], subset.obsm['spatial'], q=1e-4),
            "fraction":fraction,
            "seed":seed,
            "k":k
        })

    # Convert to DataFrame
    distance_df = pd.DataFrame(distance_data)

    end = time.time()
    print(f"Sampling and distance computation completed in {end - start:.2f} seconds.")

    return sampled_data, distance_df

def compute_distances(adata, knn=5, beta=2, platform='merfish', seed=2024, num_pcs=20, fraction=0.1):
    # Handle sparse matrix conversion
    if issparse(adata.X):
        data = adata.X.toarray()
    else:
        data = adata.X

    # Build weights matrix
    weights = RASP.build_weights_matrix(adata, n_neighbors=knn, beta=beta, platform=platform)

    # Perform PCA using fbpca
    from fbpca import pca
    U, s, Vt = pca(adata.X, k=num_pcs)
    X_dimred = U[:, :num_pcs] * s[:num_pcs]

    # Apply weights to PCA results
    smoothed_pca = weights @ csr_matrix(X_dimred)

    # Define number of samples for sketching
    N = int(fraction * adata.X.shape[0])

    # GeoSketch sampling
    sketch_index = gs(smoothed_pca.toarray(), N, seed=seed, replace=False)
    spatial_geo_sub = adata[sketch_index]

    # SCSampler sampling
    res = scsampler(smoothed_pca.toarray(), fraction=fraction, random_state=seed, copy=True)
    sub_index = res[1]
    scsampler_sketch_coords = adata[sub_index].copy()

    # Compute distances using partial_hausdorff_distance function
    scsampler_transcriptomic_distance = partial_hausdorff_distance(
        adata.X.toarray(), scsampler_sketch_coords.X.toarray(), q=1e-4)
    scsampler_spatial_distance = partial_hausdorff_distance(
        adata.obsm['spatial'], scsampler_sketch_coords.obsm['spatial'], q=1e-4)

    geo_transcriptomic_distance = partial_hausdorff_distance(
        adata.X.toarray(), spatial_geo_sub.X.toarray(), q=1e-4)
    geo_spatial_distance = partial_hausdorff_distance(
        adata.obsm['spatial'], spatial_geo_sub.obsm['spatial'], q=1e-4)

    # Store results in a dataframe with separate rows
    results = pd.DataFrame({
        'method': ['scsampler_RASP', 'geo_RASP'],
        'transcriptomic_distance': [scsampler_transcriptomic_distance, geo_transcriptomic_distance],
        'spatial_distance': [scsampler_spatial_distance, geo_spatial_distance],
        'fraction': [fraction, fraction],
        'seed': [seed, seed],
        'num_pcs': [num_pcs, num_pcs],
        'beta': [beta, beta],
        'knn':[knn,knn]
    })

    return results


def leverage_score_only_redo(adata,fraction = 0.1,seed = None,k=20): 

    # sample based on full gene set leverage score as the probabilites: 
    adata_leverage_gene_sub = probability_sample_adata(adata, fraction=fraction,score_column='gene_score', random_state=seed)
    trans_distance=partial_hausdorff_distance(adata.X.toarray(), adata_leverage_gene_sub.X.toarray(), q=1e-4)
    coord_distance = partial_hausdorff_distance(adata.obsm['spatial'], adata_leverage_gene_sub.obsm['spatial'], q=1e-4)

    #coord leverage score is a bit difficult to impliment, wait for now. 
    result = pd.DataFrame({
        'method':['leverage'],
        'transcriptomic_distance':[trans_distance],
        'spatial_distance':[coord_distance],
        'fraction':[fraction],
        'seed':[seed],
        'k':[k]
        
    })
    return result

    


In [3]:
#lets plot cell type proportions: 
directory = "/dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/data"

adata = sc.read_h5ad(os.path.join(directory,"region0_0824.h5ad"))

In [32]:
sc.pp.highly_variable_genes(adata,n_top_genes = 227)
highly_variable_genes=adata.var_names[adata.var['highly_variable']==True].tolist()

In [21]:
adata

AnnData object with n_obs × n_vars = 43038 × 228
    obs: 'fov', 'volume', 'min_x', 'max_x', 'min_y', 'max_y', 'ExpressionCounts', 'n_counts', 'leiden', 'Cell_Type', 'Cluster', 'gene_score'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'dispersions', 'dispersions_norm'
    uns: 'Cell_Type_colors', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'spatial', 't-test', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [33]:

sc.tl.score_genes(adata,gene_list = highly_variable_genes, score_name = 'gene_score')

In [34]:
result=leverage_score_only_redo(adata,fraction = 0.1,seed = 0,k=20)

building KDTree
KDTree took 0.0059 seconds to build
query tree
distance calculation took: 19.8858 seconds
building KDTree
KDTree took 0.0011 seconds to build
query tree
distance calculation took: 0.0154 seconds


In [26]:
result

,method,transcriptomic_distance,spatial_distance,fraction,seed,k
0,leverage,6.559145,126.658381,0.1,0,20


In [35]:
k = 20
fraction = 0.1


# Define the output file path where results will be stored
output_file = f"/dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/leverage_redo/leverage_sketch_{fraction}.csv"

# Initialize an empty DataFrame to gather all results
all_results = pd.DataFrame()

# Loop through 10 iterations, changing the seed each time
for i in range(10):
    print(f"Iteration {i+1}/10")

    # Call the compute_distances function with varying seeds
    distance_df = leverage_score_only_redo(adata,fraction = fraction,seed = i,k = k)

    # Append the newly computed DataFrame to the all_results DataFrame
    all_results = pd.concat([all_results, distance_df], ignore_index=True)

    # Save the concatenated results to a CSV file
    all_results.to_csv(output_file, index=False)
    print(f"Saved iteration {i+1} results to {output_file}")

# Indicate that the loop has completed
print("Loop completed. All results saved.")

Iteration 1/10
building KDTree
KDTree took 0.0080 seconds to build
query tree
distance calculation took: 19.8293 seconds
building KDTree
KDTree took 0.0011 seconds to build
query tree
distance calculation took: 0.0149 seconds
Saved iteration 1 results to /dartfs-hpc/rc/lab/F/FrostH/members/igingerich/public_data/mouse_MERFISH_data/sketching_test/leverage_redo/leverage_sketch_0.1.csv
Iteration 2/10
building KDTree
KDTree took 0.0068 seconds to build
query tree


KeyboardInterrupt: 